# Transferir datos de unidades  espaciales no anidadas

Para esta clase utilzaremos dos cartografías con unidades espaciales no superpuestas para trasnferir datos de unas a otras utilizando el porcentaje de superposición como ponderador.

Utilizaremos para ello el operador espacial `union`. Para más información sobre operadores espaciales pueden cheque la documentación de [Geopandas](http://geopandas.org/set_operations.html) o [Shapely](https://shapely.readthedocs.io/en/stable/manual.html#object.union)

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#leer la cartografía de comunas
com = gpd.read_file('../carto/clase_4/comunas-rar/comunas.shp')

#leer los datos de distritos escolares
dis = gpd.read_file('../carto/clase_4/distritos-escolares-rar/distritos_escolares.shp')

#chequear que tengan la misma proyeccion
print('mismos crs?:',dis.crs==com.crs)

#hacer un plot para observar la cartografia
fig,ax = plt.subplots(1,figsize=(8,8)) 
dis.plot(ax=ax,facecolor='grey',alpha=1)
com.plot(ax=ax,facecolor='white',edgecolor = 'black',alpha=0.1,linewidth=5);

## Data ejemplo

In [ ]:
from shapely.geometry import Polygon, Point, LineString

polys1 = gpd.GeoSeries([Polygon([(0,0), (2,0), (2,2), (0,2)])])

polys2 = gpd.GeoSeries([Polygon([(1,1), (3,1), (3,3), (1,3)])])


df1 = gpd.GeoDataFrame({'geometry': polys1, 'letra':['A']})
df2 = gpd.GeoDataFrame({'geometry': polys2, 'letra':['B']})

ax = df1.plot(color='red');

df2.plot(ax=ax, color='green', alpha=0.5);

In [ ]:
unido = gpd.overlay(df1, df2, how='union')
unido

In [ ]:
unido = unido.fillna('')
unido['union'] = unido.letra + unido.letra_2
unido

In [ ]:
unido.plot(column='union',alpha=0.5)

In [ ]:
BC = pd.DataFrame({'letra':['C','D'],'geometry':[LineString([Point(0,4),Point(4,0)]),LineString([Point(1,1),Point(3,3)])]})
BC = gpd.GeoDataFrame(BC,geometry='geometry')
BC.geometry = BC.geometry.buffer(1)
BC.plot()

In [ ]:
unido2 = gpd.overlay(unido,BC,how='union')
unido2 = unido2.fillna('')
unido2['union'] = unido2.letra + unido2.letra_2 + unido2.letra_3
unido2

In [ ]:
unido2.plot(column='union',alpha=0.5)

## Datos reales

In [ ]:
#seleccionar columnas de interes, cambiarle los nombres y el formato 
com = com.reindex(columns=['COMUNAS','AREA','geometry'])
com.columns = ['COMUNA','AREA_COMUNA','geometry']
com.COMUNA = com.COMUNA.map(lambda x: str(int(x)))
com.head(2)

In [ ]:
#obtener los datos de promedio ponderado de NBI por comuna
nbi = pd.read_csv('../data/nbi_x_comuna.csv')
nbi.COMUNA = nbi.COMUNA.map(str)
nbi.head()

In [ ]:
#pasar los datos a la cartografía de las comunas y chequear si hay datos faltantes
com = com.merge(nbi,on = 'COMUNA',how='inner')
print('Comunas sin datos:',com.NBI_prom.isnull().sum())
com.head()

In [ ]:
#seleccionar y renombrar columnas de los distritos escolars
dis = dis.reindex(columns=['ROMANO','AREA','geometry'])
dis.columns = ['DISTRITO','AREA_DISTRITO','geometry']
dis.head(2)

In [ ]:
#realizar la union y observar los datos
unidas = gpd.overlay(com,dis,how='union')
unidas.tail(2)

### Caso ejemplo

In [ ]:
#tomemos un caso de ejemplo el distrito escolar 6
fig,ax = plt.subplots(1,figsize=(8,8)) 
dis.loc[dis.DISTRITO == 'VI'].plot(ax=ax,facecolor='red',alpha=1)
com.plot(ax=ax,facecolor='white',edgecolor = 'black',alpha=0.1,linewidth=5)

In [ ]:
#crear un geodataframe del distrito 6
distrito6 = unidas.loc[unidas.DISTRITO == 'VI',:].copy()
#calcular el area de las unidades espaciales obtenidas
distrito6['area_union']=distrito6.geometry.area
distrito6

Observamos que se superpone con las comunas 5, 3 y 4. Vemos atributos de los dataset originales, como el area del distrito y el area de la comuna. También hay una parte del distrito que no se superpone con ninguna comuna

In [ ]:
#obtenemos los pesos o ponderadores como la proporción del area de las nuevas unidades espaciales 
#en realción al area de la unidad de interés, es decir los distritos 
distrito6['peso'] = round(distrito6.area_union/distrito6.AREA_DISTRITO,2)
distrito6.head()

Vemos que la unidad espacial que no se superpone con ninguna Comuna tiene un peso despreciable. En teoría esto no debería suceder,pero puede ser que la cartografía no este perfectamente alineada y haya intersticios vacíos.

In [ ]:
distrito6.loc[:,['COMUNA','NBI_prom','peso']]

In [ ]:
#chequear que los pesos den 1
distrito6.peso.sum()

In [ ]:
#eliminar los distritos sin comunas
distrito6 = distrito6.dropna()

In [ ]:
#obtener un NBI del distrito
sum(distrito6.NBI_prom * distrito6.peso)

In [ ]:
#al ver los datos originales, se observa que el nbi debía estar en algun valor entre el del NBI de las comunas 5 y 3 
# que son las que tenían mayores pesos
com.loc[com.COMUNA.isin(['4','3','5']),['COMUNA','NBI_prom']]

### Aplicación a todos los datos

In [ ]:
#calcular el area de todas las unidades nuevas 
unidas['area_union'] = unidas.geometry.area
#calcular los pesos
unidas['peso'] = unidas.area_union / unidas.AREA_DISTRITO

unidas.head(2)

In [ ]:
#explorar los pesos de las areas que no tienen comuna: todos son muy pequeños cercanos a 0
unidas.loc[(unidas.COMUNA.isnull()),'peso'].describe()

In [ ]:
#vemos donde estan los null en comuna: en los márgenes
f,ax = plt.subplots(1,figsize=(8,8))
unidas.loc[(unidas.COMUNA.isnull()) & (unidas.area_union > 1000),:].plot(color = 'red',ax=ax)
com.plot(ax=ax,color='grey',alpha=0.1)
ax.set_axis_off()

In [ ]:
#eliminar las zonas sin comuna ya que no inciden en el analisis
unidas = unidas.loc[unidas.COMUNA.notnull(),:]

In [ ]:
#crar una función que multiplique el vector del valor de NBI por el vector del peso de ese registro y lo sume
def prom_pondera(fila):
    return sum(fila.NBI_prom * fila.peso)

In [ ]:
tabla = unidas.reindex(columns=['DISTRITO','NBI_prom','peso']).groupby('DISTRITO').agg(prom_pondera)
tabla = tabla.reindex(columns = ['NBI_prom'])
tabla = tabla.reset_index()
tabla.head()

In [ ]:
dis.shape

In [ ]:
tabla.shape

In [ ]:
# Realizar un merge para pasar los datos a la cartografía de distritos
# cuando los nombres de la columna que une son iguales y ambas tablas tienen la misma cantidad de registros
#el merge es muy sencilo:
dis = dis.merge(tabla)

In [ ]:
f,ax = plt.subplots(1,figsize=(8,8))
dis.plot(ax=ax,column='NBI_prom',legend=True,scheme='Quantiles',k=5,cmap='Spectral_r')
ax.set_axis_off()
ax.set_title('NBI por Distrito escolar');

In [ ]:
#podemos crear los campos que identifiquen la union con un nombre
unidas = unidas.fillna('')
unidas['union'] = unidas.COMUNA + ' - ' + unidas.DISTRITO
unidas.head(30)